In [1]:
%matplotlib inline
%load_ext Cython 
import numpy as np
import matplotlib.pyplot as plt
import pickle
import json

In [2]:
def save_simulation_info(simulation_info,filename):
    with open(filename, 'wb') as handle:
        pickle.dump(simulation_info, handle, protocol=pickle.HIGHEST_PROTOCOL)
        
def load_simulation_info(filename):
    with open(filename,'rb') as handle:
        simulation_info=pickle.load(handle)
    
    return simulation_info

In [3]:
%%cython
cimport cython
from cython.parallel import prange
from libc.math cimport sqrt, pow, exp, asin, cos, sin
import numpy as np
cimport numpy as np

@cython.boundscheck(False)
@cython.cdivision(True)
@cython.nonecheck(False)
@cython.wraparound(False)

cdef class Metrics:

    cdef int Np
    
    def __init__(self, Np):
        
        self.Np=Np
    
    cpdef Particle_Cluster_ID (self, double [:] r, double cluster_dist):

        cdef int Np=self.Np
        cdef int i,j,k 
        cdef double Temp_A,Temp_B, dx, dy, dr

        cdef double [:] Particle_Cluster_ID = np.zeros(Np)
        
        for i in prange (Np, nogil=True):
            for j in range(i,Np):
                dx = r[i]-r[j]
                dy = r[i+Np]-r[j+Np]
                dr = sqrt(dx*dx+dy*dy)

                if dr <= cluster_dist and i != j:
                    Temp_A=Particle_Cluster_ID[i]
                    Temp_B=Particle_Cluster_ID[j]
                    Particle_Cluster_ID[i] = i+1
                    Particle_Cluster_ID[j] = i+1
                    for k in range(Np):
                        if (Particle_Cluster_ID[k] == Temp_A or Particle_Cluster_ID[k] == Temp_B) and Particle_Cluster_ID[k] != 0:
                            Particle_Cluster_ID[k] = i+1

        return Particle_Cluster_ID
    

    cpdef Cluster_ID_Info (self, double [:] r, double [:] v,  double diam, double [:] Particle_Cluster_ID,
                           double [:] Cluster_ID):

        cdef int Nc, i,j
        cdef int Np=self.Np
        cdef double dx, dy, dr, x_center, y_center, radius_gyration, num_particles

        cdef double [:] Num_Particles
        cdef double [:] X_Center
        cdef double [:] Y_Center
        cdef double [:] Radius
        cdef double [:] Radius_Gyration

        Nc=int(len(Cluster_ID))
        Num_Particles=np.zeros(Nc)
        X_Center=np.zeros(Nc)
        Y_Center=np.zeros(Nc)
        Radius=np.zeros(Nc)
        Radius_Gyration=np.zeros(Nc)

        for i in prange(Nc,nogil=True):
            x_center=0
            y_center=0
            num_particles=0
            radius_gyration=0
            
            for j in range(Np):
                if Particle_Cluster_ID[j]==Cluster_ID[i]:
                    num_particles +=1
                    x_center += r[j]
                    y_center += r[j+Np]

            Num_Particles[i] += num_particles
            X_Center[i] += x_center/num_particles
            Y_Center[i] += y_center/num_particles
            Radius[i] += sqrt(num_particles)*diam/2
            
            for j in range(Np):
                if Particle_Cluster_ID[j]==Cluster_ID[i]:
                    dx= r[j]-X_Center[i]
                    dy= r[j+Np]-Y_Center[i]
                    dr=sqrt(dx*dx+dy*dy)
                    radius_gyration += dr*dr
                    
            Radius_Gyration[i]+= sqrt(radius_gyration/num_particles)

        return Num_Particles, X_Center, Y_Center, Radius, Radius_Gyration
    

    cpdef Mechanism (self, double [:] r, double [:] p, double [:] v, double [:] Particle_Cluster_ID, 
                     double [:] Cluster_ID, double [:] Num_Particles, double [:] X_Center, double [:] Y_Center,
                     double [:] Radius, double cutoff_factor, double min_cluster_particles):

        cdef double dx, dy, dr, vx, vy, vdotr, edotr, speed, eff_radius
        cdef double G_r, G_t, gr_max, gt_max, gr, gt
        cdef double G_r_abs, G_t_abs, gr_max_abs, gt_max_abs, gr_abs, gt_abs
        cdef int i, j, Nc, counter, counter_global, calculate
        
        cdef int Np=self.Np

        Nc=int(len(Cluster_ID))
        G_r=0
        G_t=0
        G_r_abs=0
        G_t_abs=0
        
        counter_global=0

        for i in prange(Np, nogil=True):
            calculate=0
            if Particle_Cluster_ID[i]==0:
                calculate=1
            else:
                for j in range(Nc):
                    if Particle_Cluster_ID[i]==Cluster_ID[j]:
                        if Num_Particles[j] < min_cluster_particles:
                            calculate=1
                        break
                        
            if calculate==1:
                gr_max=0
                gt_max=0
                gr_max_abs = 0
                gt_max_abs = 0
                counter = 0
                for j in range(Nc):
                    eff_radius = Radius[j]*cutoff_factor
                    if Num_Particles[j]>=min_cluster_particles:
                        dx = X_Center[j]-r[i]
                        dy = Y_Center[j]-r[i+Np]
                        dr = sqrt(dx*dx+dy*dy) + 0.0001
                        if dr <= eff_radius:
                            counter = 1
                            vdotr=v[i]*dx+v[i+Np]*dy
                            edotr=p[i]*dx+p[i+Np]*dy
                            speed=sqrt(v[i]*v[i]+v[i+Np]*v[i+Np])
                            
                            gr=edotr/(dr)
                            gt=vdotr/(dr*speed)    
                            gr_abs=sqrt(edotr*edotr)/(dr)
                            gt_abs=sqrt(vdotr*vdotr)/(dr*speed)
                            
                            if gr_abs > gr_max_abs:
                                gr_max=gr
                                gr_max_abs=gr_abs
                            if gt_abs > gt_max_abs:
                                gt_max=gt
                                gt_max_abs=gt_abs

                G_r += gr_max
                G_t += gt_max
                G_r_abs += gr_max_abs
                G_t_abs += gt_max_abs
            
                counter_global += counter

        
        G_r = G_r/(counter_global+0.0001)
        G_t = G_t/(counter_global+0.0001)
        G_r_abs = G_r_abs/(counter_global+0.0001)
        G_t_abs = G_t_abs/(counter_global+0.0001)
        

        return G_r, G_t , G_r_abs, G_t_abs, 


    cpdef Clustering_Alignment_Mechanism (self, double [:] r, double [:] p, double [:] Particle_Cluster_ID,
                                          double [:] Cluster_ID, double [:]  Num_Particles,
                                          double neighbor_cutoff, double min_cluster_particles):

        cdef int i, j, k, Nc
        cdef double dx, dy, dr, P_rel, temp, Nk, pi, counter
        
        cdef int Np=self.Np
        
        Nc=int(len(Cluster_ID))
        counter=0
        P_rel=0
        
        for k in prange(Nc, nogil=True):
            if Num_Particles[k]>= min_cluster_particles:
                for i in range(Np):
                    if Particle_Cluster_ID[i] == Cluster_ID[k]:
                        Nk=0
                        temp=0
                        pi=0
                        for j in range(Np):
                            if Particle_Cluster_ID[i]==Particle_Cluster_ID[j] and i != j:
                                dx = r[i]-r[j]
                                dy = r[i+Np]-r[j+Np]
                                dr =sqrt(dx*dx+dy*dy)
                                if dr < neighbor_cutoff:
                                    temp += p[i]*p[j]+p[i+Np]*p[j+Np]
                                    Nk +=1

                        pi += sqrt(temp*temp)/(Nk+0.0001)
                        P_rel += pi
                        counter+=1

        P_rel = P_rel/(counter+0.0001)

        return P_rel

    
    cpdef Clustering_Spatial_Arrangement(self, double [:] r, double [:] Particle_Cluster_ID, 
                                       double [:] Cluster_ID, double [:]  Num_Particles,
                                       double neighbor_cutoff, double min_cluster_particles):

        cdef int i, j, k, Nc
        cdef double dx, dy, dr, Q_rel, alpha, cos_temp, sin_temp, cos_sum, sin_sum, Nk, q_abs,counter, ratio
        
        cdef int Np=self.Np
        
        Nc=int(len(Cluster_ID))
        counter=0
        Q_rel=0
        
        for k in prange(Nc, nogil=True):
            if Num_Particles[k]>=min_cluster_particles:
                for i in range(Np):
                    if Particle_Cluster_ID[i] == Cluster_ID[k]:
                        Nk=0
                        cos_temp=0
                        sin_temp=0
                        cos_sum=0
                        sin_sum=0
                        q_abs=0
                        for j in range(Np):
                            if Particle_Cluster_ID[i]==Particle_Cluster_ID[j] and i != j:
                                dx = r[j]-r[i]
                                dy = r[j+Np]-r[i+Np]
                                dr =sqrt(dx*dx+dy*dy)
                                if dr <= neighbor_cutoff:
                                    ratio=sqrt((dy/dr)*(dy/dr))
                                    if dx>0 and dy>0:
                                        alpha=asin(ratio)
                                    elif dx <0 and dy>0:
                                        alpha= 3.14 - asin(ratio)
                                    elif dx <0 and dy<0:
                                        alpha=3.14 + asin(ratio)
                                    else:
                                        alpha=2*3.14-asin(ratio)
                                    cos_sum += cos(6*alpha)
                                    sin_sum += sin(6*alpha)
                                    Nk +=1

                        cos_temp += cos_sum/(Nk+0.0001)
                        sin_temp += sin_sum/(Nk+0.0001)

                        q_abs += cos_temp*cos_temp+sin_temp*sin_temp

                        Q_rel += q_abs
                        counter+=1

        Q_rel = Q_rel/(counter+0.0001)

        return Q_rel
    
    cpdef Global_Spatial_Arrangement(self, double [:] r, double neighbor_cutoff ):

        cdef int i, j, k, Nc
        cdef double dx, dy, dr, Q_global, alpha, cos_temp, sin_temp, cos_sum, sin_sum, Nk, q_abs, counter, ratio
        
        cdef int Np=self.Np

        counter=0
        Q_global=0
        
        for i in prange(Np, nogil=True):
            Nk=0
            cos_temp=0
            sin_temp=0
            cos_sum=0
            sin_sum=0
            q_abs=0
            for j in range(Np):
                    dx = r[j]-r[i]
                    dy = r[j+Np]-r[i+Np]
                    dr =sqrt(dx*dx+dy*dy)
                    if dr <= neighbor_cutoff and i != j:
                        ratio=sqrt((dy/dr)*(dy/dr))
                        if dx>0 and dy>0:
                            alpha= asin(ratio)
                        elif dx <0 and dy>0:
                            alpha= 3.14 - asin(ratio)
                        elif dx <0 and dy<0:
                            alpha=3.14 + asin(ratio)
                        else:
                            alpha=2*3.14-asin(ratio)
                        cos_sum += cos(6*alpha)
                        sin_sum += sin(6*alpha)
                        Nk +=1

            cos_temp += cos_sum/(Nk+0.0001)
            sin_temp += sin_sum/(Nk+0.0001)

            q_abs += cos_temp*cos_temp+sin_temp*sin_temp

            Q_global += q_abs
            counter+=1

        Q_global = Q_global/(counter+0.0001)

        return Q_global
    
    cpdef Number_Inner_Particles (self, double [:] r, double neighbor_cutoff):

        cdef int i, j
        cdef double dx, dy, dr, counter, number_inner
        
        cdef int Np=self.Np

        number_inner=0
        
        for i in range(Np):
            counter=0
            for j in range(Np):
                    dx = r[j]-r[i]
                    dy = r[j+Np]-r[i+Np]
                    dr =sqrt(dx*dx+dy*dy)
                    if dr <= neighbor_cutoff and i != j:
                        counter += 1
                        
            if counter > 6:
                number_inner += 1
                    

        return number_inner

In [4]:
def Run_Metrics(filename_info, filename_results, time_start, time_end, timestep, 
                cluster_dist=4.0, mechanism_cutoff_factor=4, 
                neighbor_cutoff=4.4, min_cluster_particles=6):

    data=load_simulation_info(filename_info)
    Np=data['Np']
    diam=data['diam']
    box_length=data['box_length']
    vp=data['vp']
    mu=2.0/(6*3.14159265359*data['visc']*diam)
    Dt=data['kbT']*mu
    Dr=3*Dt/(diam*diam)
    
    data=load_simulation_info(filename_results)
    times=data['times']
    
    metrics=Metrics(Np)
    
    incret=times[1]-times[0]
    index_incret=int(timestep/incret)
    index_start=int(time_start/incret)
    index_end=int(time_end/incret)
    
    Times=[]
    Cluster_Distribution=[]
    Cluster_Radius=[]
    Cluster_Radius_Gyration=[]
    Cluster_X_Center=[]
    Cluster_Y_Center=[]
    Number_Inner_Particles=[]
    Orientation_Mechanism=[]
    Velocity_Mechanism=[]
    Orientation_Mechanism_Absolute=[]
    Velocity_Mechanism_Absolute=[]
    Rel_P=[]
    Rel_Q=[]
    Global_Q=[]
    
    for j in range(index_start,index_end,index_incret):
        
        r=data['X'][j][ :2*Np]
        p=data['X'][j][2*Np: ]
        v=data['dXdt'][j][ :2*Np]
        dpdt=data['dXdt'][j][2*Np: ]
        
        time=j*incret
        
        print('Time: ',time)
        
        Particle_Cluster_ID = metrics.Particle_Cluster_ID (r, cluster_dist)
        Particle_Cluster_ID = np.asarray(Particle_Cluster_ID)
        
        Cluster_ID=list(set(Particle_Cluster_ID))
        Cluster_ID.remove(0)
        Cluster_ID=np.array(Cluster_ID)
        
        
        Num_Particles, X_Center, Y_Center, Radius, Radius_Gyration=metrics.Cluster_ID_Info(r,v, diam, 
                                                                                           Particle_Cluster_ID, 
                                                                                           Cluster_ID)
        
        Num_Particles=np.asarray(Num_Particles)
        X_Center=np.asarray(X_Center)
        Y_Center=np.asarray(Y_Center)
        Radius=np.asarray(Radius)
        Radius_Gyration=np.asarray(Radius_Gyration)

        G_r, G_t , G_r_abs, G_t_abs= metrics.Mechanism (r, p, v, Particle_Cluster_ID, Cluster_ID, Num_Particles,
                                                        X_Center, Y_Center, Radius_Gyration,  mechanism_cutoff_factor, 
                                                        min_cluster_particles)
        
        
        P_rel= metrics.Clustering_Alignment_Mechanism (r, p, Particle_Cluster_ID, Cluster_ID, Num_Particles,
                                                       neighbor_cutoff, min_cluster_particles)
        
        
        Q_rel= metrics.Clustering_Spatial_Arrangement(r, Particle_Cluster_ID, Cluster_ID, Num_Particles,
                                                     neighbor_cutoff, min_cluster_particles)

        
        Q_global=metrics.Global_Spatial_Arrangement(r, neighbor_cutoff)
        
        Num_Inner_Particles=metrics.Number_Inner_Particles(r, neighbor_cutoff)

        
        Times.append(time)
        
        Cluster_Distribution.append(Num_Particles)
        Cluster_X_Center.append(X_Center)
        Cluster_Y_Center.append(Y_Center)
        Cluster_Radius.append(Radius)
        Cluster_Radius_Gyration.append(Radius_Gyration)

        Number_Inner_Particles.append(Num_Inner_Particles)
        
        Orientation_Mechanism.append(G_r)
        Velocity_Mechanism.append(G_t)
        Orientation_Mechanism_Absolute.append(G_r_abs)
        Velocity_Mechanism_Absolute.append(G_t_abs)
        
        Rel_P.append(P_rel)
        Rel_Q.append(Q_rel)
        Global_Q.append(Q_global)

        
    metric_results={'Times':Times, 
                    'Cluster_Distribution':Cluster_Distribution,
                    'Cluster_X_Center':Cluster_X_Center,
                    'Cluster_Y_Center':Cluster_Y_Center,
                    'Cluster_Radius':Cluster_Radius,
                    'Cluster_Radius_Gyration':Cluster_Radius_Gyration,
                    'Number_Inner_Particles':Number_Inner_Particles,
                    'Orientation_Mechanism':Orientation_Mechanism,
                    'Velocity_Mechanism':Velocity_Mechanism,
                    'Orientation_Mechanism_Absolute': Orientation_Mechanism_Absolute,
                    'Velocity_Mechanism_Absolute': Velocity_Mechanism_Absolute, 
                    'Rel_P':Rel_P,
                    'Rel_Q':Rel_Q,
                    'Global_Q':Global_Q, 
                    }
    
    return metric_results
    
    
    

In [5]:
filename_base='Linear_Flow_Run_'
run_index=[1,2,3,4,5,6,7,8,9,10]
time_start=0
time_end=2700
timestep=5

for i in run_index:
    filename_info=filename_base+str(i)+'.pickle'
    filename_results=filename_base+str(i)+'_results.pickle'

    metric_results= Run_Metrics(filename_info, filename_results, time_start, time_end, timestep, 
                                cluster_dist=4.4, mechanism_cutoff_factor=4, 
                                neighbor_cutoff=4.4, min_cluster_particles=7)

    filename=filename_base+str(i)+'_metrics.pickle'
    save_simulation_info(metric_results,filename)

Time:  0.0
Time:  5.0
Time:  10.0
Time:  15.0
Time:  20.0
Time:  25.0
Time:  30.0
Time:  35.0
Time:  40.0
Time:  45.0
Time:  50.0
Time:  55.0
Time:  60.0
Time:  65.0
Time:  70.0
Time:  75.0
Time:  80.0
Time:  85.0
Time:  90.0
Time:  95.0
Time:  100.0
Time:  105.0
Time:  110.0
Time:  115.0
Time:  120.0
Time:  125.0
Time:  130.0
Time:  135.0
Time:  140.0
Time:  145.0
Time:  150.0
Time:  155.0
Time:  160.0
Time:  165.0
Time:  170.0
Time:  175.0
Time:  180.0
Time:  185.0
Time:  190.0
Time:  195.0
Time:  200.0
Time:  205.0
Time:  210.0
Time:  215.0
Time:  220.0
Time:  225.0
Time:  230.0
Time:  235.0
Time:  240.0
Time:  245.0
Time:  250.0
Time:  255.0
Time:  260.0
Time:  265.0
Time:  270.0
Time:  275.0
Time:  280.0
Time:  285.0
Time:  290.0
Time:  295.0
Time:  300.0
Time:  305.0
Time:  310.0
Time:  315.0
Time:  320.0
Time:  325.0
Time:  330.0
Time:  335.0
Time:  340.0
Time:  345.0
Time:  350.0
Time:  355.0
Time:  360.0
Time:  365.0
Time:  370.0
Time:  375.0
Time:  380.0
Time:  385.0
Time:  3